<a href="https://colab.research.google.com/github/ANGADJEET/ANGADJEET/blob/main/boards/stm32f746g_disco_tensorflow_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Renode](https://dl.antmicro.com/projects/renode/renode.svg)](https://renode.io)

[![Run in Google Colab](https://img.shields.io/badge/-Run%20in%20Google%20colab-%23007ded?logo=google-colab&logoColor=white&style=for-the-badge)](https://colab.research.google.com/github/antmicro/test-colabs/blob/main/boards/stm32f746g_disco_tensorflow_lite_micro.ipynb) [![View ipynb](https://img.shields.io/badge/-View%20ipynb%20source-%23007ded?logo=jupyter&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/stm32f746g_disco_tensorflow_lite_micro.ipynb) [![View Python source](https://img.shields.io/badge/-View%20Python%20source-%23007ded?logo=python&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/stm32f746g_disco_tensorflow_lite_micro.py)

## Install requirements

In [1]:
! pip install -q git+https://github.com/antmicro/renode-colab-tools.git
! pip install -q git+https://github.com/antmicro/renode-run.git
! pip install -q git+https://github.com/antmicro/pyrenode3.git
! renode-run download --renode-variant dotnet-portable

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Downloaded 72.52MB / 72.52MB (time elapsed: 0:00:05.193925)...
Download finished!
Renode is already present in /root/.config/renode/renode-run.download/dotnet-portable/renode-1.15.3+20250720git2309db7fd


## Start Renode

In [2]:
import os
from renode_run import get_default_renode_path
from renode_run.utils import RenodeVariant

os.environ['PYRENODE_RUNTIME'] = 'coreclr'
os.environ['PYRENODE_BIN'] = get_default_renode_path(variant=RenodeVariant.DOTNET_PORTABLE)

from pyrenode3.wrappers import Emulation, Monitor, TerminalTester, LEDTester
from Antmicro.Renode.Peripherals.UART import UARTBackend
from Antmicro.Renode.Analyzers import LoggingUartAnalyzer
from System import String

currentDirectory = os.getcwd()
emulation = Emulation()
monitor = Monitor()
emulation.BackendManager.SetPreferredAnalyzer(UARTBackend, LoggingUartAnalyzer)

Renode found in /root/.config/renode/renode-run.download/dotnet-portable/renode-1.15.3+20250720git2309db7fd/renode


## Setup a script

In [3]:
%%writefile script.resc
logFile $ORIGIN/tensorflow_renode.log True

$name?="stm32f746g_disco"
$bin?=$ORIGIN/STM32F746G-DISCO.elf
$repl?=$ORIGIN/stm32f7.repl

using sysbus
mach create $name

machine LoadPlatformDescription $repl


showAnalyzer usart1

usart1 RecordToAsciinema $ORIGIN/tensorflow-asciinema


macro reset
"""
    sysbus LoadELF $bin
"""

runMacro $reset

Writing script.resc


## Run the sample

In [4]:
monitor.execute_script(currentDirectory + "/script.resc")
machine = emulation.get_mach("stm32f746g_disco")
terminalTester = TerminalTester(machine.sysbus.usart1, 15)

terminalTester.WaitFor(String("Running PerfTest on \"network\" with random inputs (16 iterations)..."), treatAsRegex=True, pauseEmulation=True)

emulation.Dispose()

## UART output

In [5]:
from renode_colab_tools import asciinema
asciinema.display_asciicast('tensorflow-asciinema')